In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("/home/cheemyubuntu/Documents/Projects/LLMS/Attorney-Case-Expert/data/raw/all_opinions.csv")

In [4]:
df.columns

Index(['author_name', 'category', 'per_curiam', 'case_name', 'date_filed',
       'federal_cite_one', 'absolute_url', 'cluster', 'year_filed', 'scdb_id',
       'scdb_decision_direction', 'scdb_votes_majority', 'scdb_votes_minority',
       'text'],
      dtype='object')

In [5]:
df.head(1)

,author_name,category,per_curiam,case_name,date_filed,federal_cite_one,absolute_url,cluster,year_filed,scdb_id,scdb_decision_direction,scdb_votes_majority,scdb_votes_minority,text
0,Justice Roberts,majority,False,McCutcheon v. Federal Election Comm'n,2014-04-02,NaN,https://www.courtlistener.com/opinion/2659301/...,https://www.courtlistener.com/api/rest/v3/clus...,2014,2013-033,1.0,5.0,4.0,There is no right more basic in our democracy ...


In [6]:
import torch
print(torch.cuda.memory_summary(device=None))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.profiler import profile, ProfilerActivity

model_id = "Qwen/Qwen2.5-7B-Instruct"
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, dtype=torch.bfloat16, device_map="auto", attn_implementation="sdpa"
).eval()

prompt = "Say hello in one sentence."
inputs = tok(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode(), profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    profile_memory=True,           # <-- tracks CUDA mem
    record_shapes=True,
    with_stack=False
) as prof:
    _ = model.generate(**inputs, max_new_tokens=64, use_cache=True, do_sample=True, num_beams=1)

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=30))
# Tip: also try sort_by="cuda_memory_usage" and "self_cpu_memory_usage"


/home/cheemyubuntu/miniconda3/envs/ada/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.04s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                    aten::empty_strided         0.17%      17.332ms         2.65%     278.161ms      65.931us       0.000us         0.00%       0.000us       0.000us           0 b           0 b      58.10 Gb      58.10 G

In [9]:
import torch, gc, time
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"  # adjust
PROMPT = "You are Qwen. Answer briefly: What is the capital of France?"

def bytes2gb(x): return f"{x/1024**3:.2f} GiB"

def report(where):
    if not torch.cuda.is_available(): 
        print(f"[{where}] CUDA not available"); return
    torch.cuda.synchronize()
    alloc = torch.cuda.memory_allocated()
    rsvd  = torch.cuda.memory_reserved()
    peak  = torch.cuda.max_memory_allocated()
    print(f"[{where}] allocated={bytes2gb(alloc)} reserved={bytes2gb(rsvd)} peak={bytes2gb(peak)}")
    print(torch.cuda.memory_summary())

# allocator hints to reduce fragmentation (set once per session if you like)
# import os; os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256,expandable_segments:True"

tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    dtype=torch.bfloat16,             # or torch.float16 on 3080 Ti
    device_map="auto",
    attn_implementation="sdpa",       # or "flash_attention_2" if installed
    low_cpu_mem_usage=True,
).eval()

print("Model dtype:", next(model.parameters()).dtype)
inputs = tok(PROMPT, return_tensors="pt").to(model.device)
print("Prompt tokens:", inputs["input_ids"].shape[-1])

# start fresh counters
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
report("after load + tokenize")

try:
    with torch.inference_mode():
        t0 = time.time()
        out = model.generate(
            **inputs,
            do_sample=True,
            num_beams=1,               # beams>1 multiplies KV cache
            use_cache=True,
            max_new_tokens=256,        # keep modest
            return_dict_in_generate=False,
            output_scores=False,
            output_attentions=False,
        )
    dt = time.time() - t0
    print("Gen time:", f"{dt:.2f}s")
    print(tok.decode(out[0], skip_special_tokens=True)[:200], "...")
    report("after generate")
except torch.cuda.OutOfMemoryError as e:
    print(">>> Caught OOM:", e)
    # Deep allocator snapshot to inspect later
    try:
        torch.cuda.memory._dump_snapshot("/tmp/torch_cuda_mem.json")
        print("Wrote /tmp/torch_cuda_mem.json (open in a trace viewer).")
    except Exception as ee:
        print("Snapshot failed:", ee)
    report("after OOM")
    # optional cleanup if you want to retry lighter settings
    # del out if 'out' in locals() else None
    # del model; gc.collect(); torch.cuda.empty_cache()

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 30.08it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


Model dtype: torch.bfloat16
Prompt tokens: 15
[after load + tokenize] allocated=9.42 GiB reserved=9.43 GiB peak=9.42 GiB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   9644 MiB |   9644 MiB |  78796 MiB |  69151 MiB |
|       from large pool |   9644 MiB |   9644 MiB |  77739 MiB |  68095 MiB |
|       from small pool |      0 MiB |      0 MiB |   1056 MiB |   1056 MiB |
|---------------------------------------------------------------------------|
| Active memory      